In [1]:
import brightway2 as bw
import bw2regional as bwr
import bw2_lcimpact
from copy import copy
import constructive_geometries
import os

In [2]:
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 2 object(s):
	EXIOBASE 3.3.17 hybrid
	biosphere3

In [4]:
bw.methods

Methods dictionary with 12 object(s):
	('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain')
	('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')
	('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'All')
	('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'Certain')
	('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'All')
	('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'Certain')
	('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All')
	('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain')
	('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All')
	('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'Certain')
	('LC-IMPACT', 'Water Use', 'Human Health', 'Average')
	('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal')

In [5]:
bwr.bw2regionalsetup('world')

/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [17]:
bwr.geocollections['world']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/regional/countries.gpkg',
 'field': 'isotwolettercode',
 'sha256': '654f8ee414416b3e2c2a2829cac2ea4a35c77cacb48d4ed1f58155f8dff17799',
 'kind': 'vector'}

We need to define the Rest-of-Worlds used in EXIOBASE. Happily, they are fixed, and few:

* **WA**:	RoW Asia and Pacific
* **WL**:	RoW America
* **WE**:	RoW Europe
* **WF**:	RoW Africa - really unfortunate acronym, as this is the code for an actual location...
* **WM**:	RoW Middle East

We start by gathering all the countries actually used:

In [6]:
cg = constructive_geometries.ConstructiveGeometries()

In [7]:
exiobase_rows = {"WA", "WL", "WE", "WF", "WM"}
exiobase_countries = {ds['location'] 
                      for ds in bw.Database("EXIOBASE 3.3.17 hybrid") 
                      if ds['location'] not in exiobase_rows}

In [8]:
exiobase_row_mapping = {
    "WA": "RoW Asia and Pacific",
    "WL": "RoW America",
    "WE": "RoW Europe",
    "WF": "RoW Africa",
    "WM": "RoW Middle East",
}

We will label the EXIOBASE Rest-of-Worlds with a) a new "geocollection", and b) more suitable names.

In [9]:
for activity in bw.Database("EXIOBASE 3.3.17 hybrid"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

To handle "Asia and Pacific", we will need to modify the base data in the `constructive_geometries` library. Luckily, this is not difficult.

In [10]:
asia_and_pacific = set(cg.data['UN-ASIA']).union(set(cg.data['UN-OCEANIA'])).difference(set(cg.data['UN-WASIA']))
cg.data['RoW Asia and Pacific'] = sorted(asia_and_pacific)
cg.locations.add('RoW Asia and Pacific')

In [23]:
exiobase_row_geometries = [
    ("RoW Africa", cg.construct_difference("RAF", exiobase_countries)),
    ("RoW America", cg.construct_difference('UN-AMERICAS', exiobase_countries)),
    ("RoW Europe", cg.construct_difference('UN-EUROPE', exiobase_countries)),
    ("RoW Middle East", cg.construct_difference('UN-WASIA', exiobase_countries)),
    ("RoW Asia and Pacific", cg.construct_difference('RoW Asia and Pacific', exiobase_countries)),
]

In [24]:
cg.write_geoms_to_file(
    "exiobase-rows",
    [x[1] for x in exiobase_row_geometries],
    [x[0] for x in exiobase_row_geometries],    
)

'exiobase-rows.gpkg'

We should inspect this output file, to make sure it is producing something reasonable.

<img src="images/exiobase-row.png">

In [11]:
bwr.geocollections['exiobase-rows'] = {
    'filepath': os.path.join(os.getcwd(), "exiobase-rows.gpkg"),
    'field': 'name'
}

In [12]:
bwr.geocollections.flush()

In [18]:
bw2_lcimpact.import_global_lcimpact()

In [19]:
bw2_lcimpact.import_regionalized_lcimpact()

/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/land_use.py:98: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/pm.py:103: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/pm.py:59: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/water.py:75: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/water.py:124: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00

Creating intersection (world, watersheds-hh)
Merging topographical faces for geocollection exiobase-rows



Total time elapsed: 00:00:02
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00

Creating intersection (world, world-topo-ecoregions)
Merging topographical faces for geocollection exiobase-rows



Total time elapsed: 00:00:02
0% [#####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Creating intersection (exiobase-rows, world-topo-ecoregions)


In [8]:
bwr.remote.calculate_intersection("exiobase-rows", "watersheds-hh")
bwr.remote.calculate_intersection("exiobase-rows", "watersheds-eq-sw-certain")
bwr.remote.calculate_intersection("exiobase-rows", "watersheds-eq-sw-all")
bwr.remote.calculate_intersection("exiobase-rows", "particulate-matter")
bwr.remote.calculate_intersection("exiobase-rows", "ecoregions")

Calculation job submitted.
Calculation job submitted.
Calculation job submitted.
Calculation job submitted.
Calculation job submitted.


In [ ]:
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
    'exiobase-rows-watersheds-eq-sw-certain'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
    'exiobase-rows-watersheds-eq-sw-all'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "particulate-matter", 
    'exiobase-rows-particulate-matter'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "ecoregions", 
    'exiobase-rows-ecoregions'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-hh", 
    'exiobase-rows-watersheds-hh'
)

In [9]:
sorted(bwr.intersections)

[('ecoregions', 'exiobase-rows'),
 ('ecoregions', 'world'),
 ('ecoregions', 'world-topo-ecoregions'),
 ('exiobase-rows', 'ecoregions'),
 ('exiobase-rows', 'particulate-matter'),
 ('exiobase-rows', 'watersheds-eq-sw-all'),
 ('exiobase-rows', 'watersheds-eq-sw-certain'),
 ('exiobase-rows', 'watersheds-hh'),
 ('exiobase-rows', 'world-topo-ecoregions'),
 ('exiobase-rows', 'world-topo-particulate-matter'),
 ('exiobase-rows', 'world-topo-watersheds-eq-sw-all'),
 ('exiobase-rows', 'world-topo-watersheds-eq-sw-certain'),
 ('exiobase-rows', 'world-topo-watersheds-hh'),
 ('particulate-matter', 'exiobase-rows'),
 ('particulate-matter', 'world'),
 ('particulate-matter', 'world-topo-particulate-matter'),
 ('watersheds-eq-sw-all', 'exiobase-rows'),
 ('watersheds-eq-sw-all', 'world'),
 ('watersheds-eq-sw-all', 'world-topo-watersheds-eq-sw-all'),
 ('watersheds-eq-sw-certain', 'exiobase-rows'),
 ('watersheds-eq-sw-certain', 'world'),
 ('watersheds-eq-sw-certain', 'world-topo-watersheds-eq-sw-certain'),

In [37]:
bwr.remote.calculate_intersection("watersheds-hh", 'exiobase-rows-topo-watersheds-hh')

Calculation job submitted.


In [41]:
bwr.remote.intersection("exiobase-rows-topo", 'exiobase-rows-topo-watersheds-hh')

AssertionError: Must intersect with exactly one geocollection

In [35]:
sorted(bwr.intersections)

[('ecoregions', 'exiobase-rows'),
 ('ecoregions', 'world'),
 ('ecoregions', 'world-topo-ecoregions'),
 ('exiobase-rows', 'ecoregions'),
 ('exiobase-rows', 'particulate-matter'),
 ('exiobase-rows', 'watersheds-eq-sw-all'),
 ('exiobase-rows', 'watersheds-eq-sw-certain'),
 ('exiobase-rows', 'watersheds-hh'),
 ('exiobase-rows', 'world-topo-ecoregions'),
 ('exiobase-rows', 'world-topo-particulate-matter'),
 ('exiobase-rows', 'world-topo-watersheds-eq-sw-all'),
 ('exiobase-rows', 'world-topo-watersheds-eq-sw-certain'),
 ('exiobase-rows', 'world-topo-watersheds-hh'),
 ('particulate-matter', 'exiobase-rows'),
 ('particulate-matter', 'world'),
 ('particulate-matter', 'world-topo-particulate-matter'),
 ('watersheds-eq-sw-all', 'exiobase-rows'),
 ('watersheds-eq-sw-all', 'world'),
 ('watersheds-eq-sw-all', 'world-topo-watersheds-eq-sw-all'),
 ('watersheds-eq-sw-certain', 'exiobase-rows'),
 ('watersheds-eq-sw-certain', 'world'),
 ('watersheds-eq-sw-certain', 'world-topo-watersheds-eq-sw-certain'),

In [33]:
bwr.geocollections['exiobase-rows-topo-watersheds-hh']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/regional/31371c7ad4d131f5b2685d8e20ad6d69bd17861030c9fb3b1feceab5990ed186.5efad708a9a51e08df8482c08c26f286b5e8b9ba2b0529d548edec8750cad776.geojson',
 'field': 'id',
 'url': 'https://pandarus.brightwaylca.org/intersection-file',
 'is intersection': True,
 'first': 'exiobase-rows-topo',
 'second': 'watersheds-hh',
 'sha256': '8b1db75357442df94fcc47d9955d16f6e40709bd0bc05ecd6008d1ab4ebb676d',
 'kind': 'vector'}

In [31]:
%debug

> /Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2regional/pandarus.py(143)handle_topographical_intersection()

ipdb> first_collections
{('world-topo-watersheds-hh', 'geocollection'), ('exiobase-rows-topo-watersheds-hh', 'geocollection')}
ipdb> q


In [21]:
import pandarus

In [22]:
bwr.geocollections

Geocollections dictionary with 12 object(s):
	ecoregions
	exiobase-rows
	particulate-matter
	watersheds-eq-sw-all
	watersheds-eq-sw-certain
	watersheds-hh
	world
	world-topo-ecoregions
	world-topo-particulate-matter
	world-topo-watersheds-eq-sw-all
	world-topo-watersheds-eq-sw-certain
	world-topo-watersheds-hh

In [29]:
bwr.topocollections

Topocollections dictionary with 2 object(s):
	exiobase-rows-topo
	world

In [28]:
bwr.geocollections['world-topo-ecoregions']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/regional/31371c7ad4d131f5b2685d8e20ad6d69bd17861030c9fb3b1feceab5990ed186.135cfcc1b3d9b8e2ccf3b04b797b8afeddd76185d870b5e9748a7c7e53da35f7.geojson',
 'field': 'id',
 'url': 'https://pandarus.brightwaylca.org/intersection-file',
 'is intersection': True,
 'first': 'world',
 'second': 'ecoregions',
 'sha256': '2de2582a6d59a912ffcb9d988beb921dc92312821170a79be9d372e2c958e30d',
 'kind': 'vector'}

In [25]:
ecoregions = bwr.geocollections['ecoregions']
ecoregions

{'filepath': '/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/data/land_use.gpkg',
 'field': 'eco_code',
 'sha256': '135cfcc1b3d9b8e2ccf3b04b797b8afeddd76185d870b5e9748a7c7e53da35f7',
 'kind': 'vector'}

In [26]:
world = bwr.topocollections['world']
world

{'geocollection': 'world',
 'filepath': '/Users/cmutel/Code/cg/constructive_geometries/data/faces.gpkg',
 'field': 'id',
 'empty': False,
 'sha256': '31371c7ad4d131f5b2685d8e20ad6d69bd17861030c9fb3b1feceab5990ed186',
 'kind': 'vector'}

In [27]:
new_map, _ = pandarus.intersect(
    world['filepath'],
    world['field'],
    ecoregions['filepath'],
    ecoregions['field'],
)

/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/pandarus/maps.py:39: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


KeyboardInterrupt: 